## 페르소나 및 가설1

장기 실업형 전공불일치 청년

[김OO / 29세 / 남성]

현재 상태
- 전문대 졸업 후 전공과 무관한 단기 아르바이트 반복
- 최근 1년 이상 정규 취업 경험 없음
- 현재 미취업 상태, 구직 활동은 지속 중

핵심 특징
- 학력은 있으나 직무 적합 경험 부족
- 이직·퇴직 경험은 있으나 경력으로 인정되기 어려움
- 공공 직업훈련 참여 경험은 없음 또는 1회 미만

주요 Pain Point
- “무엇을 배워야 취업이 되는지”에 대한 정보 부족
- 반복되는 미취업으로 구직 의욕 저하
- 훈련 참여 시 실제 취업으로 이어질지에 대한 불신

데이터로 표현 가능한 변수
- 만연령
- 교육정도코드 / 교육계열코드
- 미취업_퇴직후미취업기간구분코드
- 구직사항_지난4주내구직여부
- 직업교육및직장체험_직업교육수혜구분코드
- 경제활동상태코드

이 페르소나로 검증하고 싶은 가설

장기 실업 상태의 전공불일치 청년은
직업훈련 참여 여부에 따라
취업 확률의 차이가 유의미하게 발생한다.


#### 데이터 구성 및 분석 구조

| 구분      | 데이터           | 역할         |
| ------- | ------------- | ---------- |
| Model A | 경제활동인구조사 연간자료 | 개인 취업 예측   |
| Model B | 청년층 부가조사      | 직업훈련 효과 설명 |

In [ ]:
# 라이브러리 로드
import os, glob, re, json
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

import joblib

import os

from datetime import datetime

import statsmodels.api as sm
import numpy as np

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 지표 계산용
from sklearn.metrics import accuracy_score, roc_auc_score



In [ ]:
# 베이스 모델 eaps_year
print(os.getcwd())

files = glob.glob("../../data/eaps_year/*.csv")

df_eaps_year = (
    pd.concat(
        [pd.read_csv(f).assign(조사연월=lambda x: x["조사연월"].astype(str))
         for f in files],
        ignore_index=True
    )
    .sort_values("조사연월")
    .reset_index(drop=True)
)

df_eaps_year


w:\jupyterprojects\학원\1차프로젝트\notebooks\okju


,동부읍면부코드,가구주관계코드,성별코드,출생연도,교육정도_학력구분코드,교육정도_계열코드,교육정도_수학구분코드,졸업연도,혼인상태코드,현재일관련사항_지난주수입목적근로여부,...,이전직장_8차직업대분류코드,이전직장_종사상지위코드,이전직장_1자리_7차직업대분류코드,조사연월,농가비농가가구구분코드,교육정도컨버젼코드,만연령,연령5세단위코드,가중값,경제활동상태코드
0,1,3,2,2004,3,1,2,0,1,2,...,0,0,0.0,202101,2,2,16,1,629052,3
1,1,1,1,1977,5,100,1,0,1,2,...,0,0,0.0,202101,2,5,43,6,739905,1
2,1,1,1,1977,5,100,1,0,1,1,...,0,0,0.0,202101,2,5,43,6,1217099,1
3,1,1,1,1977,5,92,1,0,2,1,...,0,0,0.0,202101,2,5,43,6,1618813,1
4,1,1,1,1977,5,92,1,0,2,1,...,0,0,0.0,202101,2,5,43,6,1618813,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3409710,1,2,2,1976,5,70,1,0,2,1,...,0,0,NaN,202511,2,5,49,7,1610202,1
3409711,1,1,1,1989,3,1,1,2008,2,1,...,0,0,NaN,202511,2,3,36,5,1260207,1
3409712,1,2,2,1976,5,70,1,0,2,2,...,0,0,NaN,202511,2,5,49,7,441059,1
3409713,2,1,1,1967,5,22,1,0,2,1,...,0,0,NaN,202511,2,5,58,9,572587,1


In [ ]:
# 보조데이터 로드
print(os.getcwd())

files = glob.glob("../../data/youth_may/*.csv")

df_youth_may = (
    pd.concat(
        [pd.read_csv(f).assign(조사연월=lambda x: x["조사연월"].astype(str))
         for f in files],
        ignore_index=True
    )
    .sort_values("조사연월")
    .reset_index(drop=True)
)

df_youth_may 


w:\jupyterprojects\학원\1차프로젝트\notebooks\okju


,조사연월,동부읍면부코드,가구주관계코드,성별코드,출생연도,교육정도_학력구분코드,교육정도_계열코드,교육정도_수학구분코드,졸업연도,혼인상태코드,...,첫직장_월평균급여액,첫직장취업연월,첫직장이직연월,첫직장퇴직사유코드,첫직장업종코드,첫직장_근로형태코드,최근직장_취업경로코드,최근직장_전공관련성코드,미취업_퇴직후미취업기간구분코드,미취업_주요활동상태코드
0,202105,1,1,1,1924,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,202105,1,3,1,2005,3,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
2,202105,1,3,1,2005,3,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
3,202105,1,3,1,2005,3,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
4,202105,1,3,1,2005,3,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286923,202505,1,1,2,1964,3,3,1,0,3,...,0,0,0,0,0,0,0,0,0,0
286924,202505,1,1,2,1964,3,3,1,0,3,...,0,0,0,0,0,0,0,0,0,0
286925,202505,1,1,2,1964,3,3,1,0,3,...,0,0,0,0,0,0,0,0,0,0
286926,202505,1,1,2,1964,3,3,1,0,2,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# 컬럼 접근 규칙
COL = {
      "연령": "만연령",
    "경제활동상태": "경제활동상태코드",
    "학력": "교육정도_학력구분코드",
    "계열": "교육정도_계열코드"
}

# 조사연월 타입 통일
df_eaps_year["조사연월"] = df_eaps_year["조사연월"].astype(str)

# (선택) 분석 대상 조사연월 범위
df_eaps_base = df_eaps_year[
    df_eaps_year["조사연월"].between("202101", "202512")
].copy()

# 청년 필터 (20~39세)
df_base = df_eaps_base[
    (df_eaps_base[COL["연령"]] >= 20) &
    (df_eaps_base[COL["연령"]] <= 39)
].copy()

In [81]:

# 타깃 변수 생성

# y = 취업여부 (팀 공통 규칙)
# 1 : 취업
# 0 : 미취업 (실업 + 쉬었음)

df_base["취업여부"] = (
    df_base[COL["경제활동상태"]] == 1 
).astype(int)


In [82]:
# 가설용 파생 변수
# 장기 실업 여부 
#print(df_base.columns)

df_base["장기실업"] = (
    df_base["구직사항_구직활동기간"] >= 12
).astype(int)

In [83]:
# 개인 모델용 CSV 저장
df_base.to_csv(
    "../../data/derived/oxju/exp01_eaps_base.csv",
    index=False
)

In [ ]:
# Model A(Logistic Regression) 예측용 : 장기실업은 취업확률을 낮춘다
# 장기실업 효과 자체를 확인(전체 청년을 대상)
FEATURES = [
    COL["연령"],
    COL["학력"],
    COL["계열"],
    "장기실업"
]

X = df_base[FEATURES]
y = df_base["취업여부"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))


ROC AUC: 0.6479026912262598


In [ ]:
# 모델 평가 지표 계산
y_pred_A = model.predict(X_test)
y_prob_A = model.predict_proba(X_test)[:, 1]

accuracy_A = float(accuracy_score(y_test, y_pred_A))
roc_auc_A = float(roc_auc_score(y_test, y_prob_A))

In [86]:
# 직업훈련 효과 분석
df_training = (
    df_youth_may
    .groupby("조사연월")
    .agg(
        훈련참여율=("직업교육및직장체험_직업교육수혜구분코드", "mean")
    )
    .reset_index()
)

df_training

,조사연월,훈련참여율
0,202105,0.280687
1,202205,0.269204
2,202305,0.261839
3,202405,0.258481
4,202505,0.256302


In [ ]:

# 청년 + 기간 필터
df_youth_base = df_youth_may[
    (df_youth_may["조사연월"].between("202101", "202512")) &
    (df_youth_may["만연령"].between(20, 39))
].copy()

# 취업여부
df_youth_base["취업여부"] = (
    df_youth_base["경제활동상태코드"] == 1
).astype(int)

# 장기실업
df_youth_base["장기실업"] = (
    (df_youth_base["경제활동상태코드"] != 1) &
    (df_youth_base["구직사항_구직활동기간"] >= 12)
).astype(int)

# 직업훈련 참여 여부 (0/1)
# → 취업 이전 훈련인지 / 이후 훈련인지 구분 불가
df_youth_base["훈련참여"] = (
    df_youth_base["직업교육및직장체험_직업교육수혜구분코드"] == 1
).astype(int)

# 
df_youth_base["장기실업X훈련"] = (
    df_youth_base["장기실업"] * df_youth_base["훈련참여"]
)



In [ ]:
# Model B(로지스틱 회귀) 훈련 참여 여부에 따라 차이가 난다
# 청년층 부가조사를 활용, 직업훈련 참여 효과 및 장기실업 관계
# sklearn 모델 학습
'''
로지스틱 선택 이유
- (취업/미취업)에 적합
- 계수 및 Odds Ratio 해석 가능
- 해석 유리
'''

FEATURES_B = [
    "만연령",
    "교육정도_학력구분코드",
    "교육정도_계열코드",
    "장기실업",
    "훈련참여",
    "장기실업X훈련"
]

Xb = df_youth_base[FEATURES_B]
yb = df_youth_base["취업여부"]

Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    Xb, yb, test_size=0.2, random_state=42, stratify=yb
)

model_b = LogisticRegression(max_iter=2000)
model_b.fit(Xb_train, yb_train)

print(
    "Model B ROC AUC:",
    roc_auc_score(yb_test, model_b.predict_proba(Xb_test)[:, 1])
)


Model B ROC AUC: 0.6407226058338124


In [89]:
# 모델(sklearn) 평가 지표 계산
y_pred_B = model_b.predict(Xb_test)
y_prob_B = model_b.predict_proba(Xb_test)[:, 1]

accuracy_B = float(accuracy_score(yb_test, y_pred_B))
roc_auc_B = float(roc_auc_score(yb_test, y_prob_B))

In [ ]:
# 차이 검증중
df_sm = df_youth_base[FEATURES_B + ["취업여부"]].dropna()

X_sm = sm.add_constant(df_sm[FEATURES_B])
y_sm = df_sm["취업여부"]

logit = sm.Logit(y_sm, X_sm)
result = logit.fit()

# 훈련 효과 변수화 (
coef_training = result.params["훈련참여"]
or_training = float(np.exp(coef_training))
p_training = float(result.pvalues["훈련참여"])

print(result.summary())

# Odds Ratio
or_table = pd.DataFrame({
    "coef": result.params,
    "OR": np.exp(result.params),
    "p_value": result.pvalues
}).sort_values("p_value")

or_table

'''
* 해당해석은 gpt에게 물어봄
* OR=1.17, p-value ≪ 0.05 이상이여야 함

만연령 : coef = 0.102 / OR = 1.11, p < 0.001 유의미

교육정도_계열코드 : OR ≈ 1.00, p < 0.001 
    유의하지만 효과가 미미
    
교육정도_학력구분코드 : OR = 0.97, p = 0.005 
    학력이 높을수록 취업 확률이 소폭 감소
    
핵심 1 — 훈련참여 : coef = 0.160, OR = 1.17, p < 0.001
    직업훈련 참여자는 미참여자 대비 취업 확률이 약 17% 높음
    
핵심 2 — 장기실업 : coef = -27.56, OR ≈ 0 추정불가(완전분리)
핵심 3 — 장기실업 × 훈련 : coef = -6.54, OR ≈ 0, p ≈ 1 추정 불가
    장기실업자 중 훈련 참여자가 너무 적거나, 장기실업자 = 거의 전부 미취업

직업훈련 참여 자체는 취업 확률을 유의미하게 높이지만,
장기실업자에게서 훈련의 ‘추가 효과’는 통계적으로 추정되지 않음

직업훈련 참여는 청년의 취업 확률을 유의미하게 증가시키는 것으로 나타났다.
그러나 장기 실업 상태 자체 및 장기 실업자에 대한 훈련의 추가적 효과는
표본 내 완전분리 문제로 인해 통계적으로 안정적으로 추정되지 않았다.
''' 


         Current function value: 0.587089
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                   취업여부   No. Observations:                62404
Model:                          Logit   Df Residuals:                    62397
Method:                           MLE   Df Model:                            6
Date:                Tue, 23 Dec 2025   Pseudo R-squ.:                 0.04752
Time:                        11:13:15   Log-Likelihood:                -36637.
converged:                      False   LL-Null:                       -38464.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -2.2733      0.078    -28.973      0.000      -2.427      -2.119
만연령             0.1019      0.002     52.693      0.000       

w:\jupyterprojects\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


'\n* 해당해석은 gpt에게 물어봄\n* OR=1.17, p-value ≪ 0.05 이상이여야 함\n\n만연령 : coef = 0.102 / OR = 1.11, p < 0.001 유의미\n\n교육정도_계열코드 : OR ≈ 1.00, p < 0.001 \n    유의하지만 효과가 미미\n\n교육정도_학력구분코드 : OR = 0.97, p = 0.005 \n    학력이 높을수록 취업 확률이 소폭 감소\n\n핵심 1 — 훈련참여 : coef = 0.160, OR = 1.17, p < 0.001\n    직업훈련 참여자는 미참여자 대비 취업 확률이 약 17% 높음\n\n핵심 2 — 장기실업 : coef = -27.56, OR ≈ 0 추정불가\n핵심 3 — 장기실업 × 훈련 : coef = -6.54, OR ≈ 0, p ≈ 1 \n    장기실업자 중 훈련 참여자가 너무 적거나, 장기실업자 = 거의 전부 미취업\n\n직업훈련 참여 자체는 취업 확률을 유의미하게 높이지만,\n장기실업자에게서 훈련의 ‘추가 효과’는 통계적으로 추정되지 않음\n\n직업훈련 참여는 청년의 취업 확률을 유의미하게 증가시키는 것으로 나타났다.\n그러나 장기 실업 상태 자체 및 장기 실업자에 대한 훈련의 추가적 효과는\n표본 내 완전분리 문제로 인해 통계적으로 안정적으로 추정되지 않았다.\n'

In [ ]:
# json 생성함수

def build_experiment_json(
    owner,
    experiment_id,
    persona,
    hypothesis,
    algorithm,
    base_features,
    hypothesis_features,
    metrics_dict,
    key_result_notes,
    policy_message
):
    """
    실험 메타 JSON 생성 (값은 변수로만 관리)
    """
    meta = {
        "owner": owner,
        "experiment_id": experiment_id,
        "persona": persona,
        "hypothesis": hypothesis,
        "algorithm": algorithm,
        "features": {
            "base_features": base_features,
            "hypothesis_features": hypothesis_features
        },
        "metrics": metrics_dict,
        "key_results": key_result_notes,
        "policy_message": policy_message,
        "created_at": datetime.now().isoformat()
    }
    return meta


In [96]:
# Model A 메타 JSON 저장
MODEL_JSON_DIR = "../../results/oxju"

meta_modelA = build_experiment_json(
    owner="kim",
    experiment_id="exp01_modelA",
    persona="장기 실업형 청년",
    hypothesis="장기실업은 취업 확률을 낮춘다",
    algorithm="LogisticRegression",
    base_features=FEATURES,
    hypothesis_features=["장기실업"],
    metrics_dict={
        "accuracy": accuracy_A,
        "roc_auc": roc_auc_A
    },
    key_result_notes={
        "note": "Model A는 취업 예측 목적의 기본 모델"
    },
    policy_message="장기실업 청년은 취업 진입 자체가 구조적으로 제한됨"
)


with open(
    os.path.join(MODEL_JSON_DIR, "exp01_modelA_youth_meta.json"),
    "w",
    encoding="utf-8"
) as f:
    json.dump(meta_modelA, f, ensure_ascii=False, indent=2)

In [ ]:
 # 모델저장 Model A
 
MODEL_DIR = "../../models/oxju"
os.makedirs(MODEL_DIR, exist_ok=True)

joblib.dump(
    model,
    os.path.join(MODEL_DIR, "exp01_modelA_eaps_logistic.pkl")
)

['../../models/oxju\\exp01_modelA_eaps_logistic.pkl']

In [97]:
# Model B 저장
# 모델 저장
joblib.dump(
    model_b,
    os.path.join(MODEL_DIR, "exp01_modelB_youth_training.pkl")
)

# 메타 JSON
base_features = [
    "만연령",
    "교육정도_학력구분코드",
    "교육정도_계열코드"
]

hypothesis_features = [
    "훈련참여",
    "장기실업",
    "장기실업X훈련"
]



meta_modelB = build_experiment_json(
    owner="kim",
    experiment_id="exp01_modelB",
    persona="장기 실업형 청년",
    hypothesis="직업훈련 참여 여부는 청년의 취업 확률을 유의미하게 증가시킨다",
    algorithm="LogisticRegression",
    base_features=base_features,
    hypothesis_features=hypothesis_features,
    metrics_dict={
        "accuracy": accuracy_B,
        "roc_auc": roc_auc_B
    },
    key_result_notes={
        "training_effect": {
            "odds_ratio": or_training,
            "p_value": p_training
        },
        "longterm_unemployed_effect": {
            "note": "장기실업 변수는 취업 여부와 거의 완전 분리되어 통계적으로 안정적인 추정이 어려움"
        }
    },
    policy_message="직업훈련은 청년 취업 확률을 유의미하게 제고함"
)



with open(
    os.path.join(MODEL_JSON_DIR, "exp01_modelB_youth_meta.json"),
    "w",
    encoding="utf-8"
) as f:
    json.dump(meta_modelB, f, ensure_ascii=False, indent=2)

#### 추가

직업훈련은 청년 전반의 취업 확률을 유의미하게 제고하나,
장기 실업자에게는 훈련 이전에 노동시장 재진입을 위한
선행 개입(상담·매칭·심리 회복)이 병행될 필요가 있다.


In [ ]:
# 필요 라이브러리 import
# 데이터 처리
import pandas as pd
import numpy as np
# 통계분석
from scipy import stats
import statsmodels.api as sm
# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns
# 경고 무시
import warnings
warnings.filterwarnings('ignore')
# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False
plt.rc('axes', unicode_minus=False)
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
|

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
import os

['.config', 'sample_data']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir('/content/drive/MyDrive/csv')

['2021_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv']

In [ ]:
df21 = pd.read_csv(
    '/content/drive/MyDrive/csv/2021_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv',
    encoding='cp949'
)
df22 = pd.read_csv(
    '/content/drive/MyDrive/csv/2022_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv',
    encoding='cp949'
)
df23 = pd.read_csv(
    '/content/drive/MyDrive/csv/2023_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv',
    encoding='cp949'
)
df24 = pd.read_csv(
    '/content/drive/MyDrive/csv/2024_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv',
    encoding='cp949'
)
df25 = pd.read_csv(
    '/content/drive/MyDrive/csv/2025_경제활동인구조사 연간자료2 (2000~)_20251222_93201.csv',
    encoding='cp949'
)

,동부읍면부코드,가구주관계코드,성별코드,출생연도,교육정도_학력구분코드,교육정도_계열코드,교육정도_수학구분코드,졸업연도,혼인상태코드,현재일관련사항_지난주수입목적근로여부,...,이전직장_종사자규모코드,이전직장_8차직업대분류코드,이전직장_종사상지위코드,조사연월,농가비농가가구구분코드,교육정도컨버젼코드,만연령,연령5세단위코드,가중값,경제활동상태코드
0,1,1,1,1925,0,0,0,0,3,2,...,0,0,0,202501,2,1,99,13,283024,3
1,1,1,1,1925,0,0,0,0,3,2,...,0,0,0,202502,2,1,100,13,280570,3
2,1,1,1,1925,0,0,0,0,3,2,...,0,0,0,202503,2,1,100,13,274340,3
3,1,1,1,1926,1,0,1,0,2,2,...,0,0,0,202502,2,1,98,13,340203,3
4,1,1,1,1926,1,0,1,0,2,2,...,0,0,0,202503,2,1,98,13,326470,3
